In [1]:
from fastai.vision.all import *
matplotlib.rc('image', cmap='Greys')


In [2]:
path = untar_data(URLs.MNIST_SAMPLE)

In [3]:
Path.BASE_PATH = path

In [4]:
path.ls()

(#3) [Path('valid'),Path('labels.csv'),Path('train')]

In [5]:

(path/'train').ls()

(#2) [Path('train/7'),Path('train/3')]

In [6]:
threes = (path/'train'/'3').ls().sorted()
sevens = (path/'train'/'7').ls().sorted()
threes


(#6131) [Path('train/3/10.png'),Path('train/3/10000.png'),Path('train/3/10011.png'),Path('train/3/10031.png'),Path('train/3/10034.png'),Path('train/3/10042.png'),Path('train/3/10052.png'),Path('train/3/1007.png'),Path('train/3/10074.png'),Path('train/3/10091.png'),Path('train/3/10093.png'),Path('train/3/10097.png'),Path('train/3/10099.png'),Path('train/3/10116.png'),Path('train/3/10125.png'),Path('train/3/10137.png'),Path('train/3/10141.png'),Path('train/3/10144.png'),Path('train/3/10155.png'),Path('train/3/10161.png')...]

In [7]:
im3_path = threes[1]
im3 = Image.open(im3_path)
im3

In [16]:
im3_t = tensor(im3)
df = pd.DataFrame(im3_t[4:18,4:20])
df
# df.style.set_properties(**{'font-size':'6pt'}).background_gradient('Greys')
tensor(im3).shape

torch.Size([28, 28])

In [19]:
seven_lists = [tensor(Image.open(o)) for o in sevens]
three_lists = [tensor(Image.open(o)) for o in threes]
# len(three_tensors),len(seven_tensors)
# seven_tensors.shape

In [20]:
stacked_sevens = torch.stack(seven_lists).float()/255
stacked_threes = torch.stack(three_lists).float()/255
stacked_threes.shape

torch.Size([6131, 28, 28])

In [21]:
print("\nFirst few elements of the first 'seven' image:")
print(stacked_sevens[0, :10, :10])  # First image, first 5 rows, first 5 columns


First few elements of the first 'seven' image:
tensor([[0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0824, 0.2000, 0.8353, 0.9961, 0.9882,
         0.9882],
        [0.0000, 0.0000, 0.0000, 0.0824, 0.6314, 0.9804, 0.9804, 0.9882, 0.9804,
         0.9804],
        [0.0000, 0.0000, 0.0000, 0.2000, 0.9804, 0.9804, 0.98

In [ ]:
mean3 = stacked_threes.mean(0)
show_image(mean3);

In [ ]:

mean7 = stacked_sevens.mean(0)
show_image(mean7);

Imagine you have three images of size 2x2:
For simplicity we took just 4 pixel values and they are not normalized

```
Image 1:

[[10, 20],
 [30, 40]]
Image 2:

[[15, 25],
 [35, 45]]
Image 3:

[[20, 30],
 [40, 50]]
```



When you stack these images into a rank-3 tensor stacked_images with shape (3, 2, 2), it looks conceptually like this:
```
[
  [[10, 20],  # Image 1
   [30, 40]],

  [[15, 25],  # Image 2
   [35, 45]],

  [[20, 30],  # Image 3
   [40, 50]]
]
```
Now, if you calculate average_image = stacked_images.float().mean(dim=0), here's what happens for each pixel location:



> Pixel at (0, 0) (top-left): The values across the images are 10, 15, and 20. The average is (10 + 15 + 20) / 3 = 15.  
Pixel at (0, 1) (top-right): The values are 20, 25, and 30. The average is (20 + 25 + 30) / 3 = 25.  
Pixel at (1, 0) (bottom-left): The values are 30, 35, and 40. The average is (30 + 35 + 40) / 3 = 35.  
Pixel at (1, 1) (bottom-right): The values are 40, 45, and 50. The average is (40 + 45 + 50) / 3 = 45.



So, the resulting average_image tensor would be:
```
[[15., 25.],
 [35., 45.]]
```
In summary:

stacked_images.mean(dim=0) calculates the mean along the first dimension (dimension 0), which represents the different images in the stack. For each fixed position in the other dimensions (height and width), it takes the average of the values across all the images at that specific location.

Therefore, the resulting average_image is a single image where each pixel's intensity is the average intensity of that same pixel location across all the input images. It's a way to see the "prototypical" intensity pattern for that class of images.

In [ ]:
def mnist_distance(a,b): return (a-b).abs().mean((-1,-2))

In [ ]:
valid_3_dist = mnist_distance(valid_3_tens, mean3)
valid_3_dist, valid_3_dist.shape

Explanation of above code

Matrix Example for the First Image
To illustrate what's happening with the .mean((-1, -2)) operation, let's consider a simplified example focusing on the first image in valid_3_tens.

Assumptions:

valid_3_tens has a shape of (1010, 28, 28).  

We're looking at valid_3_tens[0], which is the first image in the tensor.  This has a shape of (28, 28).

mean3 also has a shape of (28, 28).



1.   Subtraction and Absolute Value.   
When we calculate (valid_3_tens - mean3).abs(), we perform element-wise subtraction between the pixel values of the first image in valid_3_tens and the corresponding pixel values in mean3.  The .abs() ensures we're dealing with the magnitude of the differences.  
Let's imagine a simplified 3x3 example (instead of 28x28) to visualize this.  Suppose the top-left portion of valid_3_tens[0] and mean3 look like this:



      valid_3_tens[0] (3x3 example):
      ```
          [[100, 110, 120],
          [130, 140, 150],
          [160, 170, 180]]

          mean3 (3x3 example):
          [[90, 100, 110],
          [120, 130, 140],
          [150, 160, 170]]

          Then, the result of (valid_3_tens[0] - mean3).abs() would be:

          Absolute Differences (3x3 example):
          [[10, 10, 10],
          [10, 10, 10],
          [10, 10, 10]]
      ```


2.   Applying .mean((-1, -2))

    Now, we apply .mean((-1, -2)) to this absolute difference matrix.

    -1 refers to the last dimension (width, size 3 in our example).

    -2 refers to the second-to-last dimension (height, size 3 in our example).

    This means we calculate the average of all the elements in the 3x3 matrix:

    Average = (10 + 10 + 10 + 10 + 10 + 10 + 10 + 10 + 10) / 9 = 10

    In the actual 28x28 case, .mean((-1, -2)) would calculate the average of all 784 (28 * 28) absolute differences.

3. Result for the First Image

    The result of this operation is a single number (a scalar).  In our 3x3 example, the result is 10.  In the real 28x28 scenario, it would be a single number representing the average absolute pixel difference between the first image in valid_3_tens and the mean3 image.  This value would then be stored in valid_3_dist[0].



```
accuracy_3s =      is_3(valid_3_tens).float() .mean()
accuracy_7s = (1 - is_3(valid_7_tens).float()).mean()

accuracy_3s,accuracy_7s,(accuracy_3s+accuracy_7s)/2
```



Why Combine the Accuracies?

The combined accuracy provides a summary measure of how well your classifier performs on a subset of your data (in this case, the digits "3" and "7").  It's useful for:

Overall Performance: Getting a general sense of how well the model distinguishes between these two classes.

Comparison: Comparing the performance of different classifiers or different versions of the same classifier.

Monitoring Training: Tracking how the accuracy on this subset changes during the training process.